In [ ]:
import os
from tqdm import tqdm
import numpy as np
import SimpleITK as sitk
import json
import torch
from torchmetrics.image.fid import FrechetInceptionDistance


In [ ]:
json_file = '/inye/dataset/ADNI3_test_metadata2.jsonl'
dataset = []
with open(json_file, "r") as f:
    for line in f:
        dataset.append(json.loads(line))

img_root = '/inye/dataset/T1_2D_slice_512/'
target_names = []
pre_names = []
for data in dataset:
    target_names.append(data['img'])
    pre_names.append(data['preImg'])

# FID

In [ ]:
def load_images_to_tensor(img_list, folder_path):
    # 초기 텐서 선언 (빈 텐서 또는 예상되는 최종 텐서의 크기에 맞는 텐서)
    tensors = torch.empty(0) # 여기서는 예제로 빈 텐서를 사용

    for filename in tqdm(img_list):

        file_path = os.path.join(folder_path, filename+".nii.gz")
        sitk_image = sitk.ReadImage(file_path)
        np_image = sitk.GetArrayFromImage(sitk_image)
        np_image = np_image[np.newaxis, :, :]
        np_image = np.concatenate((np_image, np_image, np_image), axis=0) # 3채널로 변환
        np_image_scaled = np.clip(np_image * 255.0, 0, 255).astype(np.uint8)
        # NumPy 배열을 PyTorch Tensor로 변환하고 새 차원 추가
        tensor = torch.from_numpy(np_image_scaled).unsqueeze(0) # 0번째 차원 추가
        
        # 0번째 차원을 기준으로 텐서 누적
        tensors = torch.cat((tensors, tensor), dim=0) if tensors.size(0) else tensor
    
    return tensors

In [ ]:
target_tensors = load_images_to_tensor(target_names, img_root)
pre_tensors = load_images_to_tensor(pre_names, img_root)

In [ ]:
target_tensors.shape, pre_tensors.shape

In [ ]:
fid = FrechetInceptionDistance(feature=64)
fid.update(target_tensors, real=True)
fid.update(pre_tensors, real=False)
fid.compute()

In [ ]:
fid = FrechetInceptionDistance(feature=192)
fid.update(target_tensors, real=True)
fid.update(pre_tensors, real=False)
fid.compute()

In [ ]:
fid = FrechetInceptionDistance(feature=768)
fid.update(target_tensors, real=True)
fid.update(pre_tensors, real=False)
fid.compute()

In [ ]:
result_path = "/inye/Inference/seed-42_strength-0.4_guidance-2.0_step-100/"

In [ ]:
result_tensors = load_images_to_tensor(target_names, result_path)

In [ ]:
result_tensors.shape

In [ ]:
fid = FrechetInceptionDistance(feature=64)
fid.update(target_tensors, real=True)
fid.update(result_tensors, real=False)
fid.compute()

In [ ]:
fid = FrechetInceptionDistance(feature=192)
fid.update(target_tensors, real=True)
fid.update(result_tensors, real=False)
fid.compute()

In [ ]:
fid = FrechetInceptionDistance(feature=768)
fid.update(target_tensors, real=True)
fid.update(result_tensors, real=False)
fid.compute()

# PSNR

In [ ]:
from torchmetrics.image import PeakSignalNoiseRatio

In [ ]:
def load_images_to_tensor(img_list, folder_path):
    # 초기 텐서 선언 (빈 텐서 또는 예상되는 최종 텐서의 크기에 맞는 텐서)
    tensors = torch.empty(0) # 여기서는 예제로 빈 텐서를 사용

    for filename in tqdm(img_list):
        
        file_path = os.path.join(folder_path, filename+".nii.gz")
        sitk_image = sitk.ReadImage(file_path)
        np_image = sitk.GetArrayFromImage(sitk_image)
        np_image = np_image[np.newaxis, :, :]
        # NumPy 배열을 PyTorch Tensor로 변환하고 새 차원 추가
        tensor = torch.from_numpy(np_image).unsqueeze(0) # 0번째 차원 추가
        
        # 0번째 차원을 기준으로 텐서 누적
        tensors = torch.cat((tensors, tensor), dim=0) if tensors.size(0) else tensor
    
    return tensors

In [ ]:
target_tensors = load_images_to_tensor(target_names, img_root)
pre_tensors = load_images_to_tensor(pre_names, img_root)

In [ ]:
target_tensors.shape, pre_tensors.shape

In [ ]:
psnr = PeakSignalNoiseRatio()
psnr(pre_tensors, target_tensors)

In [ ]:
result_tensors = load_images_to_tensor(target_names, result_path)

In [ ]:
psnr = PeakSignalNoiseRatio()
psnr(result_tensors, target_tensors)

# SSIM

In [ ]:
from torchmetrics.image import StructuralSimilarityIndexMeasure

In [ ]:
def load_images_to_tensor(img_list, folder_path):
    # 초기 텐서 선언 (빈 텐서 또는 예상되는 최종 텐서의 크기에 맞는 텐서)
    tensors = torch.empty(0) # 여기서는 예제로 빈 텐서를 사용

    for filename in tqdm(img_list):
        file_path = os.path.join(folder_path, filename+".nii.gz")
        sitk_image = sitk.ReadImage(file_path)
        np_image = sitk.GetArrayFromImage(sitk_image)
        np_image = np_image[np.newaxis, :, :]

        # NumPy 배열을 PyTorch Tensor로 변환하고 새 차원 추가
        tensor = torch.from_numpy(np_image).unsqueeze(0) # 0번째 차원 추가
        
        # 0번째 차원을 기준으로 텐서 누적
        tensors = torch.cat((tensors, tensor), dim=0) if tensors.size(0) else tensor

    return tensors

In [ ]:
target_tensors = load_images_to_tensor(target_names, img_root)
pre_tensors = load_images_to_tensor(pre_names, img_root)

In [ ]:
target_tensors.shape, pre_tensors.shape

In [ ]:
ssim = StructuralSimilarityIndexMeasure(data_range=1.0)
ssim(pre_tensors, target_tensors)

In [ ]:
result_tensors = load_images_to_tensor(target_names, result_path)

In [ ]:
ssim = StructuralSimilarityIndexMeasure(data_range=1.0)
ssim(result_tensors, target_tensors)

# LPIPS

In [ ]:
from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity

In [ ]:
def load_images_to_tensor(img_list, folder_path):
    # 초기 텐서 선언 (빈 텐서 또는 예상되는 최종 텐서의 크기에 맞는 텐서)
    tensors = torch.empty(0) # 여기서는 예제로 빈 텐서를 사용

    for filename in tqdm(img_list):
        file_path = os.path.join(folder_path, filename+".nii.gz")
        sitk_image = sitk.ReadImage(file_path)
        np_image = sitk.GetArrayFromImage(sitk_image)
        np_image = np_image[np.newaxis, :, :]
        np_image = np.concatenate((np_image, np_image, np_image), axis=0)

        # NumPy 배열을 PyTorch Tensor로 변환하고 새 차원 추가
        tensor = torch.from_numpy(np_image).unsqueeze(0).float() # 0번째 차원 추가
        
        # 0번째 차원을 기준으로 텐서 누적
        tensors = torch.cat((tensors, tensor), dim=0) if tensors.size(0) else tensor

    return tensors